# Day 9: Sensor Boost
This adds a couple more features to the Day 7 Intcode computer, mostly around the Relative Base Offset concept.

Import `defaultdict` to make the input memory array more growable.

In [149]:
from collections import defaultdict

In [150]:
# Opcode IDs
ADD = 1
MULTIPLY = 2
INPUT = 3
OUTPUT = 4
JUMP_IF_TRUE = 5
JUMP_IF_FALSE = 6
LESS_THAN = 7
EQUALS = 8
RELATIVE_BASE_OFFSET = 9
HALT = 99

In [151]:
OP_NAMES = {
    ADD: 'Add',
    MULTIPLY: 'Multiply',
    INPUT: 'Input',
    OUTPUT: 'Output',
    JUMP_IF_TRUE: 'Jump If True',
    JUMP_IF_FALSE: 'Jump If False',
    LESS_THAN: 'Less Than',
    EQUALS: 'Equals',
    RELATIVE_BASE_OFFSET: 'Relative Base Offset',
    HALT: 'Halt',
}

In [152]:
# Opcode param lengths
NUM_PARAMS = {
    ADD: 3,
    MULTIPLY: 3,
    INPUT: 1,
    OUTPUT: 1,
    JUMP_IF_TRUE: 2,
    JUMP_IF_FALSE: 2,
    LESS_THAN: 3,
    EQUALS: 3,
    RELATIVE_BASE_OFFSET: 1,
    HALT: 0,
}

We now add a new parameter mode: `RELATIVE_MODE`.

In [153]:
# Opcode param modes
POSITION_MODE = 0
IMMEDIATE_MODE = 1
RELATIVE_MODE = 2

We also add a new post-`Instruction` action called `ACTION_RELATIVE_BASE`. This will be used when the `RelativeBaseOffset` instruction finishes, and the global value of the relative base needs to be updated in the `Program` class for the next `Instructions` to use. 

In [154]:
ACTION_NEXT = 0
ACTION_JUMP = 1
ACTION_OUTPUT = 2
ACTION_RELATIVE_BASE = 3
ACTION_HALT = 99

We also modify the `Instruction` class to have an attribute called `relative_base`. This is used when reading or writing in relative mode, which is now implemented in the `get_val()` and `put_val()` methods, respectively.

In [155]:
class Instruction:
    def __init__(self, opcode, length, params, modes, relative_base, memory):
        self.opcode = opcode
        self.length = length
        self.params = params
        self.modes = modes
        self.relative_base = relative_base
        self.memory = memory
        
    def get_val(self, param_index):
        if self.modes[param_index] == IMMEDIATE_MODE:
            return self.params[param_index]
        elif self.modes[param_index] == POSITION_MODE: 
            return self.memory[self.params[param_index]]
        elif self.modes[param_index] == RELATIVE_MODE:
            return self.memory[self.params[param_index] + self.relative_base]
    
    def put_val(self, param_index, value):
        if self.modes[param_index] == POSITION_MODE:
            self.memory[self.params[param_index]] = value
        elif self.modes[param_index] == RELATIVE_MODE:
            self.memory[self.params[param_index] + self.relative_base] = value
        
    def return_control(self, action = ACTION_NEXT, value = None):
        return [action, value]

In [156]:
class Add(Instruction):
    def call(self):
        val0 = self.get_val(0)
        val1 = self.get_val(1)
        self.put_val(2, val0 + val1)
        return self.return_control()

In [157]:
class Multiply(Instruction):
    def call(self):
        val0 = self.get_val(0)
        val1 = self.get_val(1)
        self.put_val(2, val0 * val1)
        return self.return_control()                     

In [158]:
class Input(Instruction):
    def call(self, input_value = None):
        self.put_val(0, input_value)
        return self.return_control()

In [159]:
class Output(Instruction):
    def call(self):
        val = self.get_val(0)
        return self.return_control(ACTION_OUTPUT, val)

In [160]:
class JumpIfTrue(Instruction):
    def call(self):
        in_val = self.get_val(0)
        out_val = self.get_val(1)
        if in_val:
            return self.return_control(ACTION_JUMP, out_val)
        return self.return_control()

In [161]:
class JumpIfFalse(Instruction):
    def call(self):
        in_val = self.get_val(0)
        out_val = self.get_val(1)
        if not in_val:
            return self.return_control(ACTION_JUMP, out_val)   
        return self.return_control()

In [162]:
class LessThan(Instruction):
    def call(self):
        val0 = self.get_val(0)
        val1 = self.get_val(1)
        out_val = 1 if val0 < val1 else 0
        self.put_val(2, out_val)
        return self.return_control()

In [163]:
class Equals(Instruction):
    def call(self):
        val0 = self.get_val(0)
        val1 = self.get_val(1)
        out_val = 1 if val0 == val1 else 0
        self.put_val(2, out_val)
        return self.return_control()

Here is the `RelativeBaseOffset` instruction. All it does is grab a value from memory and return the `relative_base` adjusted by that value.

In [164]:
class RelativeBaseOffset(Instruction):
    def call(self):
        val = self.get_val(0)
        return self.return_control(ACTION_RELATIVE_BASE, self.relative_base + val)

In [165]:
class Halt(Instruction):
    def call(self):
        return self.return_control(ACTION_HALT)

In [166]:
OPS = {
    ADD: Add,
    MULTIPLY: Multiply,
    INPUT: Input,
    OUTPUT: Output,
    JUMP_IF_TRUE: JumpIfTrue,
    JUMP_IF_FALSE: JumpIfFalse,
    LESS_THAN: LessThan,
    EQUALS: Equals,
    RELATIVE_BASE_OFFSET: RelativeBaseOffset,
    HALT: Halt,
}

In [167]:
def create_instruction(memory, memory_pointer, relative_base):
    encoded_opcode = memory[memory_pointer]
    opcode = int(str(encoded_opcode)[-2:])
    
    num_params = NUM_PARAMS[opcode]
    
    reverse_modes = str(encoded_opcode)[:-2][::-1]
    modes = [int(char) for char in reverse_modes] + [0] * (num_params - len(reverse_modes))
    
    params = [memory[i] for i in range(memory_pointer + 1, memory_pointer + num_params + 1)]
    
    return OPS[opcode](opcode, num_params + 1, params, modes, relative_base, memory)

The only real difference to the `Program` class is again the addition of `relative_base`, and the `ACTION_RELATIVE_BASE` logic for when the `RelativeBaseOffset` instruction is encountered. This action is the same as the standard `ACTION_NEXT`, except it also updates the `relative_base` attribute of the `Program`.

In [168]:
class Program:
    def __init__(self, initial_memory):
        self.memory = initial_memory.copy()
        self.memory_pointer = 0
        self.relative_base = 0
        
    def run(self, input_values = []):
        input_counter = 0
        while(1):
            instruction = create_instruction(self.memory, self.memory_pointer, self.relative_base)
            if instruction.opcode == INPUT:
                action, value = instruction.call(input_values[input_counter])
                input_counter += 1
            else:
                action, value = instruction.call()
            
            if action == ACTION_NEXT:
                self.memory_pointer += instruction.length
            elif action == ACTION_JUMP:
                self.memory_pointer = value
            elif action == ACTION_HALT:
                break
            elif action == ACTION_OUTPUT:
                print(value)
                self.memory_pointer += instruction.length
            elif action == ACTION_RELATIVE_BASE:
                self.relative_base = value
                self.memory_pointer += instruction.length

One of the new requirements is that the `memory` we keep track of needs to be able to be arbitrarily expanded as the program executes. To do this we convert the data structure of the `memory` from a list to a default dict, where the indexes are the keys.

In [173]:
def to_dictionary(data):
    dd = defaultdict(int)
    for i, val in enumerate(data):
        dd[i] = val
    return dd

Let's run some tests. This program's output should be a copy of the program itself! WTF 

In [174]:
data = [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]
data_dict = to_dictionary(data)
Program(data_dict).run()

109
1
204
-1
1001
100
1
100
1008
100
16
101
1006
101
0
99


This program should make a giant integer. This is to make sure that our program can handle big ints.

In [175]:
data = [1102,34915192,34915192,7,4,7,99,0]
data_dict = to_dictionary(data)
Program(data_dict).run()

1219070632396864


One more test, for old times sake.

In [176]:
data = [104,1125899906842624,99]
data_dict = to_dictionary(data)
Program(data_dict).run()

1125899906842624


## Part 1:
Run the program with input `1`. This takes about 6ms to run. Not bad!

In [177]:
with open('day_09.txt') as f:
    data = [int(token) for token in f.readlines()[0].strip().split(',')]

In [178]:
%%time
data_dict = to_dictionary(data)
Program(data_dict).run([1])

2457252183
CPU times: user 2.21 ms, sys: 1.82 ms, total: 4.03 ms
Wall time: 6.05 ms


## Part 2:
Run the same program, but with input `2`. This one takes longer, but still just 2.1 seconds. I guess we coded this puppy efficiently enough!

In [172]:
%%time
data_dict = to_dictionary(data)
Program(data_dict).run([2])

70634
CPU times: user 2.08 s, sys: 7.82 ms, total: 2.09 s
Wall time: 2.1 s
